In [4]:
# Standard library imports
import asyncio
import os
import re
import warnings
from pprint import pprint

# Suppress warnings
warnings.filterwarnings("ignore")

# Third-party library imports
import bs4

# langchain imports
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_functions_agent,
    create_openai_tools_agent
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.pydantic_v1 import BaseModel, Field
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language
)
from langchain.tools import (
    BaseTool,
    StructuredTool,
    tool,
    retriever as create_retriever_tool  # Assuming you meant to import a function or module named `retriever`
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# langchain_community imports
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS

# langchain_openai imports
from langchain_openai import (
    ChatOpenAI,
    OpenAIEmbeddings
)

# Utility imports
from util_semantic_search import SemanticSearch
from util_example_based_prompt import ExampleBasedPromptFormatter

# Credentials (assuming this is a local file containing API keys or similar)
from credential import api_key


In [8]:
from credential import api_key
def question_html_generator(question: str,api_key=api_key):
    api_key = api_key
    csv_path = "Question_Embedding_20240128.csv"
    embedding_model = "text-embedding-ada-002"
    llm_model = "gpt-3.5-turbo-0125"
    n_examples = 3

    # Assuming SemanticSearch and other utilities are defined and work as expected
    semantic_search_instance = SemanticSearch(
        csv_path=csv_path,
        embedding_column_name="question",  # Adjust based on actual CSV column name
        embedding_engine=embedding_model,
        api_key=api_key
    )
    
    def extract_examples(question_str: str):
        return semantic_search_instance.extract_examples(
            input_string=question_str,
            search_column="question",
            output_column="question.html",
            n_examples=n_examples
        )

    examples_list = extract_examples(question)
    base_template = "Generate an HTML code based on the following examples."
    prompt = ExampleBasedPromptFormatter.run(examples_list, base_template) + f"\nNew question input: {question}. Delimit the generated HTML with ```insert_code_here```."

    llm = ChatOpenAI(model=llm_model, api_key=api_key, temperature=0)
    output_parser = StrOutputParser()
    chain = llm | output_parser
    html_generated = chain.invoke(prompt)

    return html_generated

# Example usage
question = "A car travels for a distance of 5mph for 30 minutes what is the distance traveled?"
html_generated = question_html_generator(question)
print(html_generated)


SyntaxError: invalid syntax (<unknown>, line 1)